# Vessel Manoeuvring Models
Many simulation model for ship manoeuvring have been developed in the field of ship hydrodynamics such as: the Abkowitz model {cite:p}`abkowitz_ship_1964` or the Norrbin model {cite:p}`norrbin_study_1960`.
This chapter will develop a general simulation model for ship manoeuvring, that can be further specified to become either the Abkowitz or Norbin model. Expressing the models on a general form is important in this research where many different models will be tested and compared.

In [1]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%reload_kedro
%config Completer.use_jedi = False  ## (To fix autocomplete)

import pandas as pd
from src.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
from src.visualization.plot import track_plots, plot, captive_plot
import kedro
import numpy as np
import os.path
import anyconfig

import matplotlib
matplotlib.rcParams["figure.figsize"] = (15,4)

from myst_nb import glue
from src.symbols import *
import src.symbols as symbols
from src.system_equations import *

from IPython.display import display, Math, Latex, Markdown
from sympy.physics.vector.printing import vpprint, vlatex

from src.parameters import df_parameters
p = df_parameters["symbol"]

# Read configs:
conf_path = os.path.join("../../conf/base/")
runs_globals_path = os.path.join(
    conf_path,
    "runs_globals.yml",
)

runs_globals = anyconfig.load(runs_globals_path)
model_test_ids = runs_globals["model_test_ids"]

join_globals_path = os.path.join(
    conf_path,
    "join_globals.yml",
)

joins = runs_globals["joins"]
join_runs_dict = anyconfig.load(join_globals_path)

globals_path = os.path.join(
    conf_path,
    "globals.yml",
)
global_variables = anyconfig.load(globals_path)



vmm_names = global_variables["vmms"]
only_joined = global_variables[
    "only_joined"
]  # (regress/predict with only models from joined runs)S

vmms = {}
for vmm_name in vmm_names:
    vmms[vmm_name] = catalog.load(vmm_name)


2022-02-18 13:56:32,543 - kedro.framework.session.store - INFO - `read()` not implemented for `SQLiteStore`. Assuming empty store.
2022-02-18 13:56:34,306 - root - INFO - ** Kedro project wPCC_pipeline
2022-02-18 13:56:34,307 - root - INFO - Defined global variable `context`, `session`, `catalog` and `pipelines`
2022-02-18 13:56:34,315 - root - INFO - Registered line magic `run_viz`
2022-02-18 13:56:35,383 - kedro.io.data_catalog - INFO - Loading data from `vmm_abkowitz` (PickleDataSet)...
2022-02-18 13:56:35,552 - kedro.io.data_catalog - INFO - Loading data from `vmm_abkowitz_simple` (PickleDataSet)...
2022-02-18 13:56:35,610 - kedro.io.data_catalog - INFO - Loading data from `vmm_martin` (PickleDataSet)...
2022-02-18 13:56:35,664 - kedro.io.data_catalog - INFO - Loading data from `vmm_linear` (PickleDataSet)...
2022-02-18 13:56:35,704 - kedro.io.data_catalog - INFO - Loading data from `vmm_martins_simple` (PickleDataSet)...
2022-02-18 13:56:35,742 - kedro.io.data_catalog - INFO - Loa

The vessel manoeuvring models can be expressed in a very general way ( {cite:p}`fossen_handbook_2021`):

In [2]:
eq_6DOF

Eq(M*\dot{\nu} + g_0 + nu*C(nu) + nu*D(nu) + g(eta), tau + tau_wave + tau_wind)

Where $\eta$ describes the position:

In [3]:
eq_eta

Eq(eta, Matrix([
[   x0],
[   y0],
[   z0],
[  phi],
[theta],
[  psi]]))

and $\nu$ is the velocities:

In [4]:
eq_nu

Eq(nu, Matrix([
[u],
[v],
[w],
[p],
[q],
[r]]))

The accelerations are denoted using the dotted notation: $\dot{\nu}$.

* $M$ is inertia matrix
* $C(\nu)$ is corriolis/centrepetal matrix as function of the velocities \nu.
* $D(\nu)$ is damping matrix as a function of vecocities \nu.
* $g(\eta)$ is a vector of generalized gravitational an buoyance forces.
* $g_0$ is static restoring forces due to ballast systems.
* $\tau$ is vector of control inputs (from rudders/propellers etc.)
* $\tau_{wind}$ is vector of wind forces
* $\tau_{wave}$ is vector of wave forces

The velocities can also include the ocean current by expressing the relative velocity $v_r$ as:

In [5]:
eq_nu_r

Eq(nu_r, nu - nu_c)

If the current is assumed to be irrotational, the angular velocities for the current is zero:

In [6]:
eq_nu_c

Eq(nu_c, Matrix([
[u_c],
[v_c],
[w_c],
[  0],
[  0],
[  0]]))

In [7]:
eq_nu_r_expanded

Eq(nu_r, Matrix([
[u - u_c],
[v - v_c],
[w - w_c],
[      p],
[      q],
[      r]]))

If the current is also assumed to be constant, this mean that the time derivative of $\nu$ and $\nu_r$ are the same, also giving the same accelerations: 

In [8]:
eq_nu_steady

Eq(\dot{\nu_r}, \dot{\nu})

When current is present the inertia as well as the corriolis matrix must be split into an added mass part (A) and a rigid body part (RB):

In [9]:
eq_M

Eq(M, M_A + M_RB)

In [10]:
eq_C

Eq(C, C_A + C_RB)

So that the model equation can be written as:

In [11]:
eq_6DOF_expanded

Eq(C_A*nu_r + C_RB*nu + D*nu_r + M_A*\dot{\nu} + M_RB*\dot{\nu} + g_0 + g(eta), tau + tau_wave + tau_wind)

This equation can be simplified for manoeuvring models by firstly only keeping surge, sway and yaw degrees of freedome. This means that both $g_0$ and $g(\eta)$ dissapears as there are no static forces for surge, sway and yaw. For the classic manoeuvring problem also forces from wind and waves are neglected, removing $\tau_{wave}$ and $\tau_{wind}$. In this paper, there are no ocean current during the studied model tests, so that the relative velocity $v_r$ can be replaced with $v_r$.

In [12]:
eq_3DOF = eq_6DOF_expanded.subs([

    (g_0,0),
    (g_function, 0),
    (tau_wave,0),
    (tau_wind,0),
    (nu_r,nu)

])
eq_3DOF

Eq(C_A*nu + C_RB*nu + D*nu + M_A*\dot{\nu} + M_RB*\dot{\nu}, tau)

And these matrices can be expressed in 3 degrees of freeddome ( {cite:p}`fossen_handbook_2021`):

In [13]:
eq_C_A

Eq(C_A, Matrix([
[                           0,             0, Y_{\dot{r}}*r + Y_{\dot{v}}*v],
[                           0,             0,                -X_{\dot{u}}*u],
[-Y_{\dot{r}} - Y_{\dot{v}}*v, X_{\dot{u}}*u,                             0]]))

In [14]:
eq_C_RB

Eq(C_RB, Matrix([
[      0, -m*r, -m*r*x_G],
[    m*r,    0,        0],
[m*r*x_G,    0,        0]]))

In [15]:
eq_M_A

Eq(M_A, Matrix([
[-X_{\dot{u}},            0,            0],
[           0, -Y_{\dot{v}}, -Y_{\dot{r}}],
[           0, -N_{\dot{v}}, -N_{\dot{r}}]]))

In [16]:
eq_M_RB

Eq(M_RB, Matrix([
[m,     0,     0],
[0,     m, m*x_G],
[0, m*x_G,   I_z]]))

The damping and control inputs from rudders and propellers are replaced with the functions $X_D(u,v,r,\delta,thrust)$, $Y_D(u,v,r,\delta,thrust)$, $N_D(u,v,r,\delta,thrust)$. Note that the measured thrust from the model tests is used as input to the models, which means that the propeller is not part of the models in this paper. Main focus is thereby on the modelling of rudder and hull forces and moments.

In [17]:
eq_D_function

Eq(tau - D(nu), Matrix([
[X_D(u, v, r, delta, thrust)],
[Y_D(u, v, r, delta, thrust)],
[N_D(u, v, r, delta, thrust)]]))

In [18]:
CD_ = (eq_C_A.rhs + eq_C_RB.rhs).doit()

The system equation can now be written as:

In [19]:
eq_system

Eq(Matrix([
[-X_{\dot{u}} + m,                    0,                    0],
[               0,     -Y_{\dot{v}} + m, -Y_{\dot{r}} + m*x_G],
[               0, -N_{\dot{v}} + m*x_G,    I_z - N_{\dot{r}}]])*Matrix([
[\dot{u}],
[\dot{v}],
[\dot{r}]]), Matrix([
[m*r**2*x_G + m*r*v + X_D(u, v, r, delta, thrust)],
[            -m*r*u + Y_D(u, v, r, delta, thrust)],
[        -m*r*u*x_G + N_D(u, v, r, delta, thrust)]]))

This equation can be rewritten to get the acceleration on the left hand side:

In [20]:
eq_acceleration_matrix_clean

Eq(Eq(\dot{\nu}, Matrix([
[\dot{u}],
[\dot{v}],
[\dot{r}]])), Matrix([
[1/(-X_{\dot{u}} + m),                         0,                         0],
[                   0,   -(-I_z + N_{\dot{r}})/S, -(-Y_{\dot{r}} + m*x_G)/S],
[                   0, -(-N_{\dot{v}} + m*x_G)/S,      -(Y_{\dot{v}} - m)/S]])*Matrix([
[m*r**2*x_G + m*r*v + X_D(u, v, r, delta, thrust)],
[            -m*r*u + Y_D(u, v, r, delta, thrust)],
[        -m*r*u*x_G + N_D(u, v, r, delta, thrust)]]))

where $S$ is a helper variable:

In [21]:
eq_S

Eq(S, -I_z*Y_{\dot{v}} + I_z*m + N_{\dot{r}}*Y_{\dot{v}} - N_{\dot{r}}*m - N_{\dot{v}}*Y_{\dot{r}} + N_{\dot{v}}*m*x_G + Y_{\dot{r}}*m*x_G - m**2*x_G**2)

A state space model for manoeuvring can now be defined with six states:

In [22]:
eq_x

Eq(\vec{x}, Matrix([
[ x_0],
[ y_0],
[\Psi],
[   u],
[   v],
[   r]]))

An transition function $f$ defines how the states changes with time:

In [23]:
eq_state_space

Eq(\dot{\vec{x}}, f(\vec{x}, u_{input}, w_{noise}))

Using geometrical relations for how $x_0$, $y_0$ and $\Psi$ depend on $u$, $v$, and $r$ and the time derivatives that was derived above: $\dot{u}$, $\dot{v}$, $\dot{r}$, the transition function can be written:

In [24]:
eq_f

Eq(f(\vec{x}, u_{input}, w_{noise}), Matrix([
[u*cos(\Psi) - v*sin(\Psi)],
[u*sin(\Psi) + v*cos(\Psi)],
[                        r],
[                  \dot{u}],
[                  \dot{v}],
[                  \dot{r}]]))

The manoeuvring simulation can now be conducted by numerical integration of the above equation. The main difference between various vessel manoeuvring models such as the Abkowitz model {cite:p}`abkowitz_ship_1964` or the Norrbin model {cite:p}`norrbin_study_1960` lies in how the hydrodynamic functions $X_D(u,v,r,\delta,thrust)$, $Y_D(u,v,r,\delta,thrust)$, $N_D(u,v,r,\delta,thrust)$ are defined. These functions cane be found in [Appendix](appendix_vmms.md).

Note that a coefficient $X_{thrust}$ has been added to the Abkowitz X equation to allow for propeller thrust as an input to the model. 

In [25]:
from wPCC_pipeline.jupyter_book import Appendix
appendix = Appendix(file_path='appendix_vmms.md', title='Vessel Manoeuvring Models')

for vmm_name, vmm in vmms.items():
    
    appendix.add_header(vmm_name, level=3)
    eqs = {'X_D':vmm.X_qs_eq,
           'Y_D':vmm.Y_qs_eq,
           'N_D':vmm.N_qs_eq,
          }
    
    for name, eq in eqs.items():
        label = f"eq_{name}_{vmm_name}"
        appendix.add_equation_multiline(eq=eq, label=label)